In [1]:
import os
import json
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_ollama import OllamaLLM
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

d:\MSc-github\msc-ir-code\ir_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.makedirs("hospital_data", exist_ok=True)

In [3]:
hospital_segments = {
  "departments": [
    {
      "topic": "Cardiology",
      "content": "The Cardiology department offers comprehensive care for patients with heart and vascular conditions. Facilities include ECG, echocardiography, coronary angiography, angioplasty, cardiac ICU, and preventive cardiology clinics."
    },
    {
      "topic": "Pediatrics",
      "content": "Our Pediatrics unit provides growth monitoring, vaccination clinics, neonatal intensive care, developmental assessments, and treatment of common to complex childhood illnesses."
    },
    {
      "topic": "Orthopedics",
      "content": "The Orthopedics department manages fractures, joint replacements, spinal surgeries, sports injuries and also runs physiotherapy services for rehabilitation."
    },
    {
      "topic": "Obstetrics & Gynecology",
      "content": "Our team provides antenatal care, labor and delivery services (including painless delivery), infertility workups, minimally invasive gynecological surgeries, and menopause care."
    },
    {
      "topic": "Neurology",
      "content": "The Neurology unit evaluates and manages strokes, epilepsy, multiple sclerosis, migraine, and neurodegenerative disorders, with 24/7 stroke response protocols."
    },
    {
      "topic": "Oncology",
      "content": "The Oncology department delivers chemotherapy, radiation therapy, surgical oncology, palliative care, and counseling support for cancer patients."
    },
    {
      "topic": "Nephrology",
      "content": "Focused on kidney health, offering dialysis services, renal transplantation support, and management of chronic kidney disease and electrolyte disorders."
    },
    {
      "topic": "Dermatology",
      "content": "Provides medical and cosmetic services including acne treatment, skin biopsies, dermatologic surgery, and laser therapies for skin rejuvenation."
    },
    {
      "topic": "Psychiatry & Mental Health",
      "content": "The Psychiatry department offers counseling, psychotherapy, de-addiction services, in-patient psychiatric care, and community mental health programs."
    },
    {
      "topic": "Radiology & Imaging",
      "content": "Our Radiology department provides diagnostic and interventional imaging: MRI, CT, ultrasound, digital X-rays, mammography and image-guided biopsies."
    }
  ],

  "services": [
    {
      "topic": "Emergency",
      "content": "A 24/7 Emergency department staffed with rapid response teams for trauma, strokes, cardiac arrests, poisoning, and accident care."
    },
    {
      "topic": "OPD",
      "content": "Outpatient services across all departments with appointment scheduling, follow-up consultations, diagnostic testing and patient education."
    },
    {
      "topic": "Inpatient Wards",
      "content": "General, semi-private and private wards with nursing care, dietary support, daily rounds by treating physicians and allied health professionals."
    },
    {
      "topic": "ICU Services",
      "content": "Multiple intensive care units including medical ICU, surgical ICU, neonatal ICU (NICU) and cardiac ICU equipped with advanced monitoring systems."
    },
    {
      "topic": "Surgical Services",
      "content": "Includes general surgery, laparoscopic procedures, trauma surgeries, transplant support, and day-care minimal invasive interventions."
    },
    {
      "topic": "Laboratory Services",
      "content": "24×7 accredited lab performing hematology, pathology, microbiology, biochemistry and histopathology tests with fast turnaround time."
    },
    {
      "topic": "Pharmacy",
      "content": "In-house pharmacy providing prescription medications, over-the-counter products, patient counseling and safe medication practices."
    },
    {
      "topic": "Rehabilitation & Physiotherapy",
      "content": "Post-operative and injury rehab, physiotherapy for orthopedic/neuro patients, occupational therapy, and speech therapy."
    },
    {
      "topic": "Preventive Health Packages",
      "content": "Master health check-ups, annual screenings, cardiology evaluations, cancer screening and lifestyle modification clinics."
    },
    {
      "topic": "Vaccination Clinic",
      "content": "Provides adult, pediatric and travel-related vaccines under guidance of trained doctors and immunization nurses."
    }
  ],

  "labs": [
    {
      "topic": "Pathology",
      "content": "Performs routine blood counts, urine analysis, tissue biopsy processing, Pap smears and bone marrow examination. Critical values are reported immediately to treating doctors."
    },
    {
      "topic": "Biochemistry",
      "content": "Analyzes blood glucose, lipid profile, liver/kidney function tests, cardiac enzymes and hormone assays with rapid automated analyzers."
    },
    {
      "topic": "Microbiology",
      "content": "Handles cultures for bacteria, fungi and viruses, antibiotic sensitivity testing, infection surveillance, and hospital hygiene audits."
    },
    {
      "topic": "Molecular Diagnostics",
      "content": "Specialized genetic and PCR-based testing for infectious diseases, cancer markers, and personalized medicine guidance."
    },
    {
      "topic": "Blood Bank & Transfusion Services",
      "content": "Provides safe blood and blood products, with cross-matching, plasma separation and platelet pheresis facilities. Runs 24/7 emergency support."
    }
  ],

  "wards": [
    {
      "topic": "General Ward",
      "content": "Economical multi-bed facility with central nursing station, shared restrooms, and round-the-clock care for stable patients."
    },
    {
      "topic": "Semi-Private Ward",
      "content": "Shared by 2–3 patients, air-conditioned rooms with attached washroom, and visiting lounge access."
    },
    {
      "topic": "Private Ward",
      "content": "Single patient rooms with attached bathrooms, television, visitor seating, higher degree of privacy and optional dietary customization."
    },
    {
      "topic": "ICU",
      "content": "Critical care unit equipped with ventilators, bedside monitors, infusion pumps, isolation facilities, and dedicated intensivists."
    },
    {
      "topic": "NICU",
      "content": "Neonatal Intensive Care Unit for premature or critically ill newborns, with incubators, phototherapy, ventilators and trained neonatologists."
    },
    {
      "topic": "Recovery & Day Care Unit",
      "content": "Short-stay facility for post-operative observation of patients undergoing minor surgeries or diagnostic procedures."
    }
  ],

  "consultants": [
    {
      "name": "Dr. Anil Mehra",
      "specialty": "Cardiology",
      "experience": "20 years",
      "qualifications": "MD (Medicine), DM (Cardiology)",
      "availability": "Mon–Sat, 10 AM – 2 PM",
      "contact": "cardiology@hospital.org"
    },
    {
      "name": "Dr. Priya Nair",
      "specialty": "Pediatrics",
      "experience": "12 years",
      "qualifications": "MD (Pediatrics), Fellowship in Neonatology",
      "availability": "Mon–Fri, 9 AM – 1 PM",
      "contact": "pediatrics@hospital.org"
    },
    {
      "name": "Dr. Suresh Iyer",
      "specialty": "Orthopedics",
      "experience": "18 years",
      "qualifications": "MS (Orthopedics), Fellowship in Joint Replacement",
      "availability": "Tue–Sat, 3 PM – 6 PM",
      "contact": "orthopedics@hospital.org"
    },
    {
      "name": "Dr. Kavita Shah",
      "specialty": "Obstetrics & Gynecology",
      "experience": "15 years",
      "qualifications": "MD (Obgyn), Diploma in Laparoscopic Surgery",
      "availability": "Mon–Sat, 11 AM – 4 PM",
      "contact": "gynecology@hospital.org"
    },
    {
      "name": "Dr. Rajesh Gupta",
      "specialty": "Neurology",
      "experience": "22 years",
      "qualifications": "DM (Neurology), Fellowship in Stroke Medicine",
      "availability": "Mon/Wed/Fri, 1 PM – 5 PM",
      "contact": "neurology@hospital.org"
    },
    {
      "name": "Dr. Neha Verma",
      "specialty": "Oncology",
      "experience": "10 years",
      "qualifications": "MD (Medicine), DM (Medical Oncology)",
      "availability": "Mon–Fri, 10 AM – 1 PM",
      "contact": "oncology@hospital.org"
    }
  ],

  "guidelines": [
    {
      "topic": "Visiting Hours",
      "content": "General wards: 10 AM – 12 PM and 5 PM – 7 PM daily. ICU timings strictly limited to 4 PM – 5 PM, only one visitor at a time."
    },
    {
      "topic": "COVID & Infection Control",
      "content": "Masks must be worn within premises. Hand sanitizers are installed at all entry points, PPE use is mandatory in isolation wards, and visitors with fever/cough are discouraged."
    },
    {
      "topic": "Patient Admission",
      "content": "Please carry valid ID proof, insurance documents (if applicable), and prior medical records. Admission is processed via Admissions Desk on ground floor."
    },
    {
      "topic": "Discharge Process",
      "content": "Discharge is authorized by treating consultant, patient summary and prescriptions are provided, and billing must be cleared before handing over documents."
    },
    {
      "topic": "Billing & Payments",
      "content": "Payments are accepted via cash, cards, or online transfers. Insurance/TPA assistance counter is functional 9 AM – 6 PM on weekdays."
    },
    {
      "topic": "Patient Rights",
      "content": "All patients are entitled to dignity, privacy, confidentiality of records, informed consent, and respectful communication from staff."
    },
    {
      "topic": "Patient Responsibilities",
      "content": "Provide accurate health information, follow prescribed treatments, respect hospital property, and maintain considerate behavior towards staff and co-patients."
    },
    {
      "topic": "Smoking & Alcohol Policy",
      "content": "Hospital campus is strictly tobacco- and alcohol-free. Violations may result in fines and reporting to authorities."
    },
    {
      "topic": "Parking & Cafeteria",
      "content": "Visitor parking is available at basement levels B1 and B2. Cafeteria operates 7 AM – 9 PM with safe, hygienic food options."
    }
  ]
}
 


In [4]:
for fname, data in hospital_segments.items():
    with open(os.path.join("hospital_data", fname), "w") as f:
        json.dump(data, f, indent=2)
 
print("✅ Hospital dataset prepared in /hospital_data")

✅ Hospital dataset prepared in /hospital_data


In [5]:
all_texts = []
 
for section, items in hospital_segments.items():   # departments, services, etc.
    for entry in items:                            # each dict inside the list
        for key, value in entry.items():           # key = "topic"/"content"/etc.
            if isinstance(value, str):             # only collect text fields
                all_texts.append(value)
 
print(len(all_texts))       # how many strings collected
print(all_texts[:10])       # preview first 10 entries

116
['Cardiology', 'The Cardiology department offers comprehensive care for patients with heart and vascular conditions. Facilities include ECG, echocardiography, coronary angiography, angioplasty, cardiac ICU, and preventive cardiology clinics.', 'Pediatrics', 'Our Pediatrics unit provides growth monitoring, vaccination clinics, neonatal intensive care, developmental assessments, and treatment of common to complex childhood illnesses.', 'Orthopedics', 'The Orthopedics department manages fractures, joint replacements, spinal surgeries, sports injuries and also runs physiotherapy services for rehabilitation.', 'Obstetrics & Gynecology', 'Our team provides antenatal care, labor and delivery services (including painless delivery), infertility workups, minimally invasive gynecological surgeries, and menopause care.', 'Neurology', 'The Neurology unit evaluates and manages strokes, epilepsy, multiple sclerosis, migraine, and neurodegenerative disorders, with 24/7 stroke response protocols.']

In [6]:
# Chunk documents
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = splitter.create_documents(all_texts)

In [7]:
# Hospital Embeddings
embedding = OllamaEmbeddings(model="nomic-embed-text")

C:\Users\User\AppData\Local\Temp\ipykernel_25220\2290740292.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(model="nomic-embed-text")


In [8]:
# !ollama pull nomic-embed-text

In [9]:
# To check whether embedder is working
out = embedding.embed_query("Test embedding for hospital")
print(len(out), out[:5])

768 [0.3504588305950165, 1.4587610960006714, -3.825359582901001, -1.4360544681549072, 1.0577150583267212]


In [10]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings
 
embedding = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")
 
vectordb = FAISS.from_documents(docs, embedding)
 
# Save to disk
vectordb.save_local("hospital_faiss_index")
 
print("✅ Hospital Knowledge Base indexed with FAISS")

C:\Users\User\AppData\Local\Temp\ipykernel_25220\1945492002.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")


✅ Hospital Knowledge Base indexed with FAISS


In [11]:
# !ollama pull phi3:mini

In [12]:
# Load Ollama LLM (adjust model name to what's available locally, e.g. llama2, mistral)

llm = OllamaLLM(model="phi3:mini")
#llm = Ollama(model="phi3:mini") 
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 1}) 

# Hospital-specific prompt template
# Create prompt template
prompt = PromptTemplate.from_template("""
You are a helpful hospital virtual assistant. Use the given hospital context to answer.

Hospital Context:
{context}

Patient Question:
{question}

Answer ONLY using the provided Hospital Context.
Keep your response short, clear, and factual (2 - 4 sentences maximum).
""")

# Create RAG chain using LCEL (LangChain Expression Language)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Use it like this:
response = rag_chain.invoke("What are the visiting hours?")
print(response)

The hospital's current visiting hours policy allows for visits from two to four visitors per patient at any time during normal business hours of 9:0 endtime daily except on weekends. Visitors are required to check in with the front desk upon arrival, and guidelines must be followed regarding hygiene practices due to health safety protocols. Please note that visiting hours may change based on individual patient needs or specific wards within the hospital.


In [13]:
docs = retriever.invoke("What are the visiting hours?")
print("Docs returned:", len(docs))
for d in docs:
    print(d.page_content[:160])

Docs returned: 1
Visiting Hours


In [14]:
%%writefile hospital_chat_app.py
# =======================================
# 🏥 Fast Hospital Assistant Chatbot
# =======================================
import streamlit as st
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

 
# ---- Load FAISS Knowledge Base ----
embedding = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")
vectordb = FAISS.load_local("hospital_faiss_index", embedding, allow_dangerous_deserialization=True)
 
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 2})
 
# ---- Streamlit UI ----
st.set_page_config(page_title="🏥 Fast Hospital Assistant Chatbot", layout="wide")
st.title("🏥 Virtual Hospital Assistant (FAISS + Ollama)")
 
# ---- Smalltalk filter ----
def is_smalltalk(msg: str) -> bool:
    greetings = ["hi", "hello", "hey", "good morning", "good evening"]
    return msg.lower().strip() in greetings
 
# ---- Model selector ----
model_choice = st.sidebar.selectbox(
    "Select Model",
    ["phi3:mini", "gemma:2b", "mistral"]
)
 
# ---- Define Ollama LLM ----
llm = OllamaLLM(
    model=model_choice,
    streaming=True,
    options={"num_ctx": 1024, "num_predict": 120, "temperature": 0.3}
)
 
# ---- Strict grounded prompt ----
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a hospital assistant AI.
 
Answer ONLY using the Hospital Context.
Keep replies short, clear, and factual (max 3 sentences).  
If the context does not contain an answer, reply:  
"I'm sorry, I don't have that information in the hospital database."
 
Hospital Context:
{context}
 
Patient Question:
{question}
 
Response:
"""
)
 
# Create RAG chain using LCEL
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
 
# ---- Memory for chat history ----
if "history" not in st.session_state:
    st.session_state["history"] = []
 
# ---- Render history ----
for role, msg in st.session_state["history"]:
    with st.chat_message("user" if role == "user" else "assistant"):
        st.markdown(msg)
 
# ---- Chat input ----
user_message = st.chat_input("Ask me about hospital services, departments, or guidelines...")
 
if user_message:
    # 1. Display user msg
    st.session_state["history"].append(("user", user_message))
    with st.chat_message("user"):
        st.markdown(user_message)
 
    # 2. Handle smalltalk separately
    if is_smalltalk(user_message) or len(user_message.split()) < 2:
        bot_reply = "👋 Hello! How can I assist you with hospital information today?"
        # Save assistant reply
        st.session_state["history"].append(("assistant", bot_reply))
        with st.chat_message("assistant"):
            st.markdown(bot_reply)
    
    else:
        # 3. Run RAG pipeline
        bot_reply = rag_chain.invoke(user_message)
        sources = retriever.invoke(user_message)  # Get sources separately

        # Show response with sources in chat message
        with st.chat_message("assistant"):
            st.markdown(bot_reply)
            with st.expander("🔍 Context from FAISS (debug)"):
                if sources:
                    for i, doc in enumerate(sources, 1):
                        st.markdown(f"**Doc {i}:** {doc.page_content}")
                else:
                    st.warning("⚠️ No hospital documents retrieved.")
        
        # 4. Save assistant reply
        st.session_state["history"].append(("assistant", bot_reply))

Overwriting hospital_chat_app.py


In [15]:
# Run this in command prompt
# !streamlit run hospital_chat_app.py